## Vistan Apartments
Few places in Chula Vista have had drones dispatched to them more frequently than the Vistan Apartments on the West side. The complex, in one of the poorest neighborhoods in Chula Vista, is populated largely by Black and Latine families, many of whom are immigrants. Over the last two years, CVPD drones have spent a total of TK hours flying above the Vistan Apartments and have been dispatched to respond to incidents ranging from indecent exposure to even an attempted kidnapping. 

Vistan Apartments resident Jesus Lopez says that one afternoon, while he was away, a man banged loudly on his front door. When his three-year-old opened it, the stranger reached in and tried to grab his kid. According to Lopez, his babysitter quickly yanked the child inside, slammed the door, and dialed 911. Within minutes, Lopez says, a police drone arrived to search for the suspect. 

The department did not find the individual, but, nevertheless, Lopez—who spoke to WIRED through his locked door, because he doesn't open it for people he doesn’t recognize anymore—says that police drones make him feel safer. 

In [1]:
import datetime
import os

import fiona
import geopandas as gpd
import geopy.distance
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
from loguru import logger

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
storage = "/Volumes/easystore/Drones"
coi=['Incident No.','Description','cfs-date','Block Location','Latitude','Longitude','drone-id']

calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

flight_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_drone-seconds-block-count-per-flight.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/mapping/cv-blocks.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks['GEOID20'] = cv_blocks['GEOID20'].apply(lambda x: x[3:])
cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [2]:
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"])
cv_flights["drone-date"] =  pd.to_datetime(cv_flights['date'] + " " + cv_flights['time_s'])
cv_flights["match"] = cv_flights["incident_id"].astype(str).apply(lambda x: x.strip().replace(' ','').replace('L','0').replace('l','0')[-5:])
calls_for_service["match"] = (
    calls_for_service["Incident No."].astype(str).apply(lambda x: x.strip().replace(' ','').replace('l','0').replace('L','0')[-5:])
)

/var/folders/4c/vsvqyby107s_6_0qp0lykjwm0000gn/T/ipykernel_4566/2413963456.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cv_flights["drone-date"] =  pd.to_datetime(cv_flights['date'] + " " + cv_flights['time_s'])


In [3]:
matches = cv_flights[cv_flights["match"].isin(calls_for_service["match"])]
matches_joined_cfs = pd.merge(matches, calls_for_service, how="left", on="match",suffixes=['-drones','-cfs'])

matches_joined_cfs = matches_joined_cfs[
    matches_joined_cfs["drone-date"].dt.date == matches_joined_cfs["cfs-date"].dt.date
]
flights_with_cfs  = matches_joined_cfs.copy()



In [4]:
vistan_flight_blocks = flight_blocks[(flight_blocks['GEOID20']=='060730125012001')]

logger.info(f"{vistan_flight_blocks['id'].drop_duplicates().shape[0]} flights passed through vistan")
logger.info(f"{vistan_flight_blocks['seconds'].astype(int).sum()/3600} hours of footage")

2024-05-24 11:42:48.362 | INFO     | __main__:<module>:3 - 312 flights passed through vistan
2024-05-24 11:42:48.363 | INFO     | __main__:<module>:4 - 8.018888888888888 hours of footage


In [5]:

vistan_flight_ids = vistan_flight_blocks['id'].drop_duplicates().values

vistan_flights = flights_with_cfs[flights_with_cfs['id'].isin(vistan_flight_ids)]
vistan_flights.to_csv('./vistan-flights.csv',index=False)


In [6]:
kidnapping

NameError: name 'kidnapping' is not defined

In [10]:
flights_with_cfs[flights_with_cfs['Incident No.'].isin(kidnapping['Incident No.'])]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,...,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,cfs-date
